In [14]:
import pandas as pd 

method = "all"
correlation_type = "pearson"

averages_raw = pd.read_csv(f"../data/page6_LinearXWASCorrelations/average_correlations/Correlations_comparisons_{method.capitalize()}_{correlation_type.capitalize()}.csv", index_col=0)
averages_raw.index = pd.MultiIndex.from_tuples(averages_raw.index.str.split("_").tolist(), names=("dimension_2", "dimension_1"))
averages = averages_raw.swaplevel().drop(columns=["Phenotypic.1", "Genetics.1"])


averages_std_raw = pd.read_csv(f"../data/page6_LinearXWASCorrelations/average_correlations/Correlations_sd_comparisons_{method.capitalize()}_{correlation_type.capitalize()}.csv", index_col=0)
averages_std_raw.index = pd.MultiIndex.from_tuples(averages_std_raw.index.str.split("_").tolist(), names=("dimension_2", "dimension_1"))
averages_std = averages_std_raw.swaplevel().drop(columns=["Phenotypic.1", "Genetics.1"])


averages.rename(index={"*": "set", "*instances01": "set_instances01", "*instances1.5x": "set_instances1.5x", "*instances23": "set_instances23"}, inplace=True)
averages.loc["Eyes"]

,Phenotypic,Genetics,Alcohol,AnthropometryBodySize,AnthropometryImpedance,ArterialStiffness,BloodBiochemistry,BloodCount,BloodPressure,BoneDensitometryOfHeel,...,medical_diagnoses_W,medical_diagnoses_X,medical_diagnoses_Y,medical_diagnoses_Z,All_All,All_Biomarkers,All_Phenotypes,All_Diseases,All_Environmental,All_Socioeconomics
dimension_2,,,,,,,,,,,,,,,,,,,,,
Average,0.132389,0.123804,0.565790,0.527136,0.280677,0.545697,0.527742,0.412816,-0.131377,0.779955,...,0.391431,0.255798,0.351273,0.638358,0.241949,0.332150,0.225497,0.469795,0.448641,0.403250
set,0.419783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
set_instances01,0.362023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
set_instances1.5x,0.174496,0.145512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
set_instances23,0.346544,0.422699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abdomen,0.201637,0.359996,0.519948,-0.159800,0.280777,0.533928,0.295374,0.297267,-0.140560,0.788262,...,0.296642,0.452474,0.317939,0.684824,0.364106,0.471152,0.199046,0.455523,0.459846,0.267391
AbdomenLiver,0.190359,0.291578,0.508306,-0.144399,0.334128,0.338553,0.270507,0.230104,-0.082528,0.795363,...,0.262738,0.388132,0.352317,0.662101,0.367321,0.493255,0.194484,0.413728,0.426304,0.259739
AbdomenPancreas,0.184584,0.451931,0.528492,-0.172670,0.306193,0.445629,0.357555,0.365947,-0.089051,0.794629,...,0.277947,0.434974,0.266554,0.629454,0.359774,0.471215,0.253845,0.425101,0.414199,0.242674
Arterial,0.148133,0.246055,0.403243,0.511789,-0.477764,0.565849,0.484495,0.421756,0.160944,0.831555,...,0.140808,0.121157,0.014887,0.502524,0.244567,0.287989,0.107985,0.307438,0.327518,0.067029
